## ДЗ №3 Двухуровневый пайплайн
#### В этой домашке вам предстоит написать с нуля двустадийную рекомендательную систему. 

#### Дата выдачи: 10.03.25

#### Мягкий дедлайн: 31.03.25 23:59 MSK

#### Жесткий дедлайн: 7.04.25 23:59 MSK

### Описание
Это творческое задание, в котором вам необходимо реализовать полный цикл построения рекомендательной системы: реализовать кандидат генераторов, придумать и собрать признаки, обучить итоговый ранкер и заинференсить модели на всех пользователей.

Вам предоставляется два набора данных: `train.csv` и `test.csv` 

In [ ]:
import numpy as np
import pandas as pd
from typing import List
from collections import defaultdict
from surprise import SVD, Dataset, Reader
from torch import nn
import torch
from torch.utils.data import Dataset, DataLoader

In [3]:
!pip3 install gdown

  Using cached gdown-5.2.0-py3-none-any.whl.metadata (5.8 kB)
  Using cached beautifulsoup4-4.13.3-py3-none-any.whl.metadata (3.8 kB)
  Using cached filelock-3.18.0-py3-none-any.whl.metadata (2.9 kB)
  Using cached soupsieve-2.6-py3-none-any.whl.metadata (4.6 kB)
  Using cached PySocks-1.7.1-py3-none-any.whl.metadata (13 kB)
Using cached gdown-5.2.0-py3-none-any.whl (18 kB)
Using cached beautifulsoup4-4.13.3-py3-none-any.whl (186 kB)
Using cached filelock-3.18.0-py3-none-any.whl (16 kB)
Using cached PySocks-1.7.1-py3-none-any.whl (16 kB)
Using cached soupsieve-2.6-py3-none-any.whl (36 kB)


In [ ]:
# скачиваем данные
# если из этой ячейки не получается, то вот ссылка на папку https://drive.google.com/drive/folders/1HT0Apm8Jft0VPLJtdBBUGu9s1M7vZcoJ?usp=drive_link

!pip3 install gdown


import gdown
# train
url = "https://drive.google.com/file/d/1-CcS22-UpTJeNcFlA0dVLrEQn8jnI0d-/view?usp=drive_link"
output = 'train.csv'
gdown.download(url, output, quiet=False)

# test
url = "https://drive.google.com/file/d/11iz3xDh0IIoEIBY0dyRSvByY3qfiT3BG/view?usp=drive_link"
output = 'test.csv'
gdown.download(url, output, quiet=False)

Downloading...
From: https://drive.google.com/file/d/1-CcS22-UpTJeNcFlA0dVLrEQn8jnI0d-/view?usp=drive_link
To: d:\DZ_RECSYS\train.csv
34.3kB [00:00, 15.7MB/s]
Downloading...
From: https://drive.google.com/file/d/11iz3xDh0IIoEIBY0dyRSvByY3qfiT3BG/view?usp=drive_link
To: d:\DZ_RECSYS\test.csv
91.1kB [00:00, 3.27MB/s]


'test.csv'



### 1 Этап. Модели первого уровня. (max 3 балла)
В этом этапе вам необходимо разделить `train` датасет на 2 части: для обучения моделей первого уровня и для их валидации. Единственное условие для разбиения – разбивать нужно по времени. Данные для обучение будем называть `train_stage_1`, данные для валидации `valid_stage_1`. Объемы этих датасетов вы определяет самостоятельно. 

Для начала нам нужно отобрать кандидатов при помощи легких моделей. Необходимо реализовать 3 типа моделей:
1. Любая эвристическая(алгоритмичная) модель на ваш выбор **(0.5 балл)**
2. Любая матричная факторизация на ваш выбор **(1 балл)**
3. Любая нейросетевая модель на ваш выбор **(1 балла)**

Не забудьте использовать скор каждой модели, как признак!



In [50]:
# Загрузка данных
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')
train_df.last_watch_dt.max()

'2021-08-12'

In [53]:
# Преобразование временных меток
train_df['weekday'] = pd.to_datetime(train_df.last_watch_dt).dt.weekday
test_df['weekday'] = pd.to_datetime(test_df.last_watch_dt).dt.weekday
train_df = train_df.sort_values('last_watch_dt')

train_stage_1 = train_df.loc[(train_df.last_watch_dt < '2021-08-06')].copy()
valid_stage_1 = train_df.loc[train_df.last_watch_dt >= '2021-08-06'].copy()

train_users = train_stage_1.user_id.unique()
valid_users = valid_stage_1.user_id.unique()
test_users = test_df.user_id.unique()


all_included = np.intersect1d(np.intersect1d(valid_users, train_users), test_users)

print('number of users which are included both in train/test data: ', all_included.shape[0])

number of users which are included both in train/test data:  52311


In [54]:
test_df.shape, train_stage_1.shape, valid_stage_1.shape

((608467, 7), (4473822, 7), (393134, 7))

In [59]:
train_grouped = train_stage_1.groupby('user_id').apply(
    lambda x: [(t1, t2, t3) for t1, t2, t3 in sorted(zip(x.item_id, 
                                                 x.last_watch_dt,
                                                 x.weekday), key=lambda x: x[1])]
).reset_index()
train_grouped.rename({0:'train_interactions'}, axis=1, inplace=True)

valid_grouped = valid_stage_1.groupby('user_id').apply(
    lambda x: [(t1, t2, t3) for t1, t2, t3 in sorted(zip(x.item_id,
                                                         x.last_watch_dt,
                                                         x.weekday), key=lambda x: x[1])]
).reset_index()
valid_grouped.rename({0:'valid_interactions'}, axis=1, inplace=True)

test_grouped = test_df.groupby('user_id').apply(
    lambda x: [(t1, t2, t3) for t1, t2, t3 in sorted(zip(x.item_id,
                                                         x.last_watch_dt,
                                                         x.weekday), key=lambda x: x[1])]
).reset_index()
test_grouped.rename({0:'test_interactions'}, axis=1, inplace=True)


joined = train_grouped.merge(valid_grouped).merge(test_grouped)
joined.head()

C:\Users\user\AppData\Local\Temp\ipykernel_19172\3262174671.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  train_grouped = train_stage_1.groupby('user_id').apply(
C:\Users\user\AppData\Local\Temp\ipykernel_19172\3262174671.py:8: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  valid_grouped = valid_stage_1.groupby('user_id').apply(
C:\Users\user\AppData\Local\Temp\ipykernel_19172\3262174671.py:15: Depreca

,user_id,train_interactions,valid_interactions,test_interactions
0,3,"[(2657, 2021-06-03, 3), (3734, 2021-07-22, 3),...","[(12192, 2021-08-11, 2), (15719, 2021-08-11, 2...","[(142, 2021-08-13, 4), (10240, 2021-08-16, 0),..."
1,17,"[(3734, 2021-08-01, 6)]","[(8618, 2021-08-06, 4), (10196, 2021-08-06, 4)]","[(9342, 2021-08-15, 6)]"
2,30,"[(4740, 2021-06-09, 2), (676, 2021-06-12, 5), ...","[(16484, 2021-08-11, 2), (3031, 2021-08-11, 2)]","[(15363, 2021-08-13, 4), (11253, 2021-08-13, 4..."
3,47,"[(13865, 2021-07-29, 3), (13361, 2021-07-29, 3...","[(12681, 2021-08-09, 0), (12846, 2021-08-11, 2)]","[(14488, 2021-08-20, 4)]"
4,53,"[(13865, 2021-07-01, 3), (7571, 2021-07-01, 3)...","[(12250, 2021-08-10, 1)]","[(16426, 2021-08-15, 6), (15810, 2021-08-17, 1..."


In [ ]:
# 1. Эвристическая модель (Popularity-based)
class TopPopular:
    def __init__(self):
        self.trained = False
    
    def fit(self, df, col='train_interactions'):
        counts = {}
        for _, row in df.iterrows():
            for item, _, _ in row[col]:
                counts[item] = counts.get(item, 0) + 1
                    
        self.recommendations = [x[0] for x in sorted(counts.items(), 
                                                   key=lambda x: -x[1])]
        self.trained = True
        
    def predict(self, df, topn=10) -> List[np.ndarray]:
        assert self.trained
        return [self.recommendations[:topn]] * len(df)
    
    def predict_score(self, user_id, item_id):
        try:
            rank = self.recommendations.index(item_id)
            return 1.0 / (rank + 1)
        except ValueError:
            return 0.5
# Инициализация и обучение моделей
my_heuristic_model = TopPopular()
my_heuristic_model.fit(joined)
joined['toppopular_recs'] = my_heuristic_model.predict(joined)

In [ ]:
# 2. Матричная факторизация (SVD)
class SVDRecommender:
    def __init__(self, n_factors=50, n_epochs=20):
        self.model = None
        self.user_map = {}
        self.item_map = {}
        self.default_score = 0.5
        self.trained = False
        
    def fit(self, df, col='train_interactions'):
        # Create mappings
        users = set()
        items = set()
        interactions = []
        
        for _, row in df.iterrows():
            user_id = row['user_id']
            users.add(user_id)
            for item_id, _, rating in row[col]:
                items.add(item_id)
                interactions.append((user_id, item_id, rating))
                
        self.user_map = {u: i for i, u in enumerate(users)}
        self.item_map = {i: j for j, i in enumerate(items)}
        
        # Prepare data for surprise
        reader = Reader(rating_scale=(0, 1))
        data = Dataset.load_from_df(pd.DataFrame(interactions, 
                                               columns=['user_id', 'item_id', 'rating']), 
                                  reader)
        trainset = data.build_full_trainset()
        
        # Train model
        self.model = SVD(n_factors=n_factors, n_epochs=n_epochs, verbose=False)
        self.model.fit(trainset)
        self.trained = True
        
    def predict(self, df, topn=10) -> List[np.ndarray]:
        assert self.trained
        all_items = list(self.item_map.keys())
        recommendations = []
        
        for _, row in df.iterrows():
            user_id = row['user_id']
            scores = []
            for item_id in all_items:
                try:
                    score = self.model.predict(self.user_map[user_id], 
                                            self.item_map[item_id]).est
                    scores.append((item_id, score))
                except:
                    continue
            
            top_items = [x[0] for x in sorted(scores, key=lambda x: -x[1])[:topn]]
            recommendations.append(top_items)
            
        return recommendations
    
    def predict_score(self, user_id, item_id):
        try:
            return self.model.predict(self.user_map[user_id], 
                                   self.item_map[item_id]).est
        except:
            return self.default_score

my_matrix_factorization = SVDRecommender()
my_matrix_factorization.fit(joined)
joined['svd_recs'] = my_matrix_factorization.predict(joined)

In [ ]:
# 1. Эвристическая модель (Popularity-based)
class PopularityRecommender:
    def __init__(self):
        self.popular_items = None
        
    def fit(self, df):
        # Ранжируем товары по количеству взаимодействий
        self.popular_items = df['item_id'].value_counts().reset_index()
        self.popular_items.columns = ['item_id', 'score']
        
    def recommend(self, user_id, n=10):
        return self.popular_items.head(n)

popular_model = PopularityRecommender()
popular_model.fit(train_stage_1)

In [29]:
# 2. Матричная факторизация (implicit ALS)
def prepare_implicit_data(df):
    user_ids = df['user_id'].astype('category').cat.codes
    item_ids = df['item_id'].astype('category').cat.codes
    interactions = csr_matrix((np.ones(len(df)), (user_ids, item_ids)))
    user_map = dict(enumerate(df['user_id'].astype('category').cat.categories))
    item_map = dict(enumerate(df['item_id'].astype('category').cat.categories))
    return interactions, user_map, item_map

interactions, user_map, item_map = prepare_implicit_data(train_stage_1)

als_model = implicit.als.AlternatingLeastSquares(
    factors=64, 
    iterations=15, 
    regularization=0.1,
    random_state=42
)
als_model.fit(interactions.T)

def als_predict(user_id, n=10):
    if user_id not in user_map.values():
        return pd.DataFrame({'item_id': [], 'als_score': []})
    internal_id = list(user_map.keys())[list(user_map.values()).index(user_id)]
    items, scores = als_model.recommend(internal_id, interactions[internal_id], N=n)
    return pd.DataFrame({
        'item_id': [item_map[i] for i in items],
        'als_score': scores
    })

d:\DZ_RECSYS\.venv\lib\site-packages\implicit\utils.py:164: ParameterWarning: Method expects CSR input, and was passed csc_matrix instead. Converting to CSR took 0.08099079132080078 seconds
  warnings.warn(
100%|██████████| 15/15 [00:37<00:00,  2.49s/it]


In [31]:
# 3. Нейросетевая модель (Neural Collaborative Filtering)
train_grouped = train_stage_1.groupby('user_id').apply(
    lambda x: [(t1, t2, t3) for t1, t2, t3 in sorted(zip(x.item_id, 
                                                 x.last_watch_dt,
                                                 x.weekday), key=lambda x: x[1])]
).reset_index()
train_grouped.rename({0:'train_interactions'}, axis=1, inplace=True)

valid_grouped = valid_stage_1.groupby('user_id').apply(
    lambda x: [(t1, t2, t3) for t1, t2, t3 in sorted(zip(x.item_id,
                                                         x.last_watch_dt,
                                                         x.weekday), key=lambda x: x[1])]
).reset_index()
valid_grouped.rename({0:'valid_interactions'}, axis=1, inplace=True)

test_grouped = test_df.groupby('user_id').apply(
    lambda x: [(t1, t2, t3) for t1, t2, t3 in sorted(zip(x.item_id,
                                                         x.last_watch_dt,
                                                         x.weekday), key=lambda x: x[1])]
).reset_index()
test_grouped.rename({0:'test_interactions'}, axis=1, inplace=True)


train_grouped.head()

AttributeError: 'DataFrame' object has no attribute 'weekday'

In [ ]:
# Генерация признаков для второго уровня
def generate_features(df):
    results = []
    
    for user_id in df['user_id'].unique():
        # Получаем рекомендации от всех моделей
        pop_recs = popular_model.predict(user_id)
        als_recs = als_predict(user_id)
        ncf_recs = ncf_predict(user_id)
        
        # Объединяем результаты
        merged = pop_recs.merge(als_recs, on='item_id', how='outer')\
                        .merge(ncf_recs, on='item_id', how='outer')
        merged['user_id'] = user_id
        results.append(merged)
    
    return pd.concat(results).fillna(0)

train_features = generate_features(train_stage_1)
valid_features = generate_features(valid_stage_1)

# Сохранение признаков
train_features.to_csv('train_features.csv', index=False)
valid_features.to_csv('valid_features.csv', index=False)

In [3]:
my_heuristic_model = # YOUR CODE HERE
my_matrix_factorization = # YOUR CODE HERE
my_neural_network = # YOUR CODE HERE 


SyntaxError: invalid syntax (3627771860.py, line 1)

Каждая модель должна уметь:
1) для пары user_item предсказывать скор релевантности (масштаб скора не важен), важно обработать случаи, когда модель не можеn проскорить пользователя или айтем, вместо этого вернуть какое-то дефолтное значение
2) для всех пользователей вернуть top-k самых релевантных айтемов (тут вам скоры не нужны)


Дополнительно можно провести анализ кандидат генератов, измерить насколько различные айтемы они рекомендуют, например с помощью таких метрик как: [Ranked based overlap](https://github.com/changyaochen/rbo) или различные вариации [Diversity](https://github.com/MaurizioFD/RecSys2019_DeepLearning_Evaluation/blob/master/Base/Evaluation/metrics.py#L289). **(1 балл)**


### 2 Этап. Генерация и сборка признаков. (max 2 балла)
Необходимо собрать минимум 10 осмысленных (`np.radndom.rand()` не подойдет) признаков, при этом:
1. 2 должны относиться только к сущности "пользователь" (например средний % просмотра фильмов у этой возрастной категории)
2. 2 должны относиться только к сущности "айтем" (например средний средний % просмотра данного фильма)
3. 6 признаков, которые показывают связь пользователя и айтема (например средний % просмотра фильмов с данным актером (айтем) у пользователей с таким же полом (пользователь)). 

### ВАЖНО!  

1. **В датасете есть колонка `watched_prct`. Ее можно использовать для генерации признаков (например сколько пользователь в среднем смотрит фильмы), но нельзя подавать в модель, как отдельную фичу, потому что она напрямую связана с target.**
2. **Все признаки должны быть собраны без дата лика, то есть если пользователь посмотрел фильм 10 августа, то признаки мы можем считать только на данных до 9 августа включительно.**


### Разбалловка
Обучение ранкера будет проходить на `valid_stage_1`, как  раз на которой мы валидировали модели, а тестировать на `test`. Поэтому есть 2 варианта сборки признаков, **реализовать нужно только 1 из них:**
1. Для обучения собираем признаки на первый день `valid_stage_1`, а для теста на первый день `test`. Например, если `valid_stage_1` начинается 5 сентября, то все признаки мы можем собирать только по 4 сентября включительно. **(1 балл)**
2. Признаки будем собирать честно на каждый день, то есть на 5 сентября собираем с начала до 4, на 6 сентября с начала до 5 и т.д. **(2 балла)**

In [ ]:
train_df_with_features = # YOUR CODE IS HERE
test_df_with_features = # YOUR CODE IS HERE


### 3 Этап. Обучение финального ранкера (max 2 балла)
Собрав все признаки из этапа 2, добавив скоры моделей из этапа 1 для каждой пары пользователь-айтем (где это возможно), пришло время обучать ранкер. В качестве ранкера можно использовать либо [xgboost](https://xgboost.readthedocs.io/en/stable/) или [catboost](https://catboost.ai/). Обучать можно как `Classfier`, так и `Ranker`, выбираем то, что лучше сработает. Обучение ранкера будет проходить на `valid_stage_1`, как  раз на которой мы валидировали модели, а тестировать на `test`, которую мы до сих пор не трогали.  Заметьте, что у нас в тесте есть холодные пользователи – те, кого не было в train и активные – те, кто был в train. Возможно их стоит обработать по отдельности (а может и нет).  
(1 балл)

После получения лучшей модели надо посмотреть на важность признаков и [shap values](https://shap.readthedocs.io/en/latest/index.html), чтобы:
1. Интерпритировать признаки, которые вы собрали, насколько они полезные
2. Проверить наличие ликов – если важность фичи в 100 раз больше, чем у всех остальных, то явно что-то не то  

(1 балл)






In [ ]:
# YOUR FIT PREDICT CODE HERE
model.fit()
model.predict()


### 4 Этап. Инференс лучшего ранкера (max 3 балла)

Теперь мы хотим построить рекомендации "на завтра", для этого нам нужно:

1. Обучить модели первого уровня на всех (train+test) данных (0.5 балла)
2. Для каждой модели первого уровня для каждого пользователя сгененировать N кандидатов (0.5 балла)
3. "Склеить" всех кандидатов для каждого пользователя (дубли выкинуть), посчитать скоры от всех моделей (0.5 балла)
4. Собрать фичи для ваших кандидатов (теперь можем считать признаки на всех данных) (0.5 балла)
5. Проскорить всех кандидатов бустингом и оставить k лучших (0.5 балла)
6. Посчитать разнообразие(Diversity) и построить график от Diversity(k) (0.5 балла)


Все гиперпараметры (N, k) определяете только Вы!

In [3]:
# YOUR CODE HERE